In [1]:
!pip install opendatasets
import opendatasets as od

In [2]:
od.download("https://www.kaggle.com/datasets/prajwalmohapatra/fire-probability-prediction-map-unstacked-data/data")

Skipping, found downloaded files in "./fire-probability-prediction-map-unstacked-data" (use force=True to force download)


In [3]:
import rasterio
import numpy as np
import os
import glob

from rasterio.warp import reproject, Resampling

# Paths
dir = '/home/swayam/projects/forest_fire_spread/fire-probability-prediction-map-unstacked-data'
base_dir = os.path.join(dir, 'dataset_unstacked')
weather_dir = os.path.join(base_dir, 'weather')
fire_mask_dir = os.path.join(base_dir, 'fire_mask')
output_dir = os.path.join(dir, 'dataset_stacked')

print(dir)
print(base_dir)
print(weather_dir)
print(fire_mask_dir)

# Static layers
slope_path = os.path.join(base_dir, 'slope', 'uk_slope_2016.tif')
aspect_path = os.path.join(base_dir, 'aspect', 'uk_aspect_2016.tif')
fuel_path = os.path.join(base_dir, 'fuel_map', 'uk_fuel_map_2020.tif')
ghsl_path = os.path.join(base_dir, 'ghsl', 'ghsl_uk_urbanmask2015.tif')

print()
print(slope_path)
print(aspect_path)
print(fuel_path)
print(ghsl_path)

# Create output directory
os.makedirs(output_dir, exist_ok=True)

/home/swayam/projects/forest_fire_spread/fire-probability-prediction-map-unstacked-data
/home/swayam/projects/forest_fire_spread/fire-probability-prediction-map-unstacked-data/dataset_unstacked
/home/swayam/projects/forest_fire_spread/fire-probability-prediction-map-unstacked-data/dataset_unstacked/weather
/home/swayam/projects/forest_fire_spread/fire-probability-prediction-map-unstacked-data/dataset_unstacked/fire_mask

/home/swayam/projects/forest_fire_spread/fire-probability-prediction-map-unstacked-data/dataset_unstacked/slope/uk_slope_2016.tif
/home/swayam/projects/forest_fire_spread/fire-probability-prediction-map-unstacked-data/dataset_unstacked/aspect/uk_aspect_2016.tif
/home/swayam/projects/forest_fire_spread/fire-probability-prediction-map-unstacked-data/dataset_unstacked/fuel_map/uk_fuel_map_2020.tif
/home/swayam/projects/forest_fire_spread/fire-probability-prediction-map-unstacked-data/dataset_unstacked/ghsl/ghsl_uk_urbanmask2015.tif


## **Step 1: Load Static Layers Once and Use as Reference**

In [4]:
# Load static layers once, outside the loop
with rasterio.open(slope_path) as s:
  slope = s.read(1, masked=True)
  ref_meta = s.meta.copy() # Reference metadata
  ref_shape = s.shape # Reference shape (height, width) (9551, 12917)
  ref_transform = s.transform
  ref_crs = s.crs

  print(ref_meta)
  # print(ref_shape)
  # print(ref_transform)
  # print(ref_crs)

with rasterio.open(aspect_path) as a:
  aspect = a.read(1, masked=True)
  # print(aspect)
  print(a.meta.copy())

with rasterio.open(fuel_path) as f:
  fuel = f.read(1, masked=True)
  # print(fuel)
  print(f.meta.copy())

with rasterio.open(ghsl_path) as g:
  ghsl = g.read(1, masked=True)
  # print(ghsl)
  print(g.meta.copy())

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 12917, 'height': 9551, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00026949458523585647, 0.0, 77.56161960922043,
       0.0, -0.00026949458523585647, 31.290207807979588)}
{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 12917, 'height': 9551, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00026949458523585647, 0.0, 77.56161960922043,
       0.0, -0.00026949458523585647, 31.290207807979588)}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 38750, 'height': 28650, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(8.983152841195215e-05, 0.0, 77.56161960922043,
       0.0, -8.983152841195215e-05, 31.29011797645117)}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 3876, 'height': 2865, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0008983152841195215, 0.0, 77.5614399461636,
       0.0, -0.0008983152841195215, 31.290117976451175

## **Step 2: Resample Fuel and GHSL Data (Relatively Safe to Run)**

In [5]:
# fuel_path = os.path.join(base_dir, 'fuel_map', 'uk_fuel_map_2020.tif')
# ghsl_path = os.path.join(base_dir, 'ghsl', 'ghsl_uk_urbanmask2015.tif')

output_fuel_path = os.path.join(base_dir, 'fuel_map', 'resampled_uk_fuel_map_2020.tif')
output_ghsl_path = os.path.join(base_dir, 'ghsl', 'resampled_ghsl_uk_urbanmask2015.tif')

In [6]:
# Function to resample a raster to match the reference
def resample_raster(src_path, ref_meta, ref_shape, ref_transform, ref_crs, output_path, resampling_method):
    with rasterio.open(src_path) as src:
        # Read the single band, handle masked arrays
        data = src.read(1, masked=True)  # Read as masked array
        if np.ma.is_masked(data):
            data = data.filled(src.nodata if src.nodata is not None else -9999)  # Fill with nodata or default
        else:
            data = data.astype(np.float32)  # Ensure consistent dtype if no mask
        
        # Initialize destination array (2D)
        resampled_data = np.zeros(ref_shape, dtype=np.float32)
        
        # Reproject to match reference
        reproject(
            source=data,
            destination=resampled_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=ref_transform,
            dst_crs=ref_crs,
            resampling=resampling_method,
            dst_nodata=-9999  # Define nodata for output
        )
        
        # Update metadata for output
        out_meta = ref_meta.copy()
        out_meta.update({
            'dtype': 'float32',  # Use float32 for consistency
            'nodata': -9999,
            'count': 1
        })
        
        # Write the resampled raster
        with rasterio.open(output_path, 'w', **out_meta) as dst:
            dst.write(resampled_data, 1)

In [7]:
# Resample fuel dataset (continuous data, use bilinear)
if not os.path.exists(output_fuel_path):
    resample_raster(fuel_path, ref_meta, ref_shape, ref_transform, ref_crs, output_fuel_path, Resampling.bilinear)

# Resample GHSL dataset (categorical data, use nearest neighbor)
if not os.path.exists(output_ghsl_path):
    resample_raster(ghsl_path, ref_meta, ref_shape, ref_transform, ref_crs, output_ghsl_path, Resampling.nearest)

print(f"Resampled fuel dataset saved to {output_fuel_path}")
print(f"Resampled GHSL dataset saved to {output_ghsl_path}")

Resampled fuel dataset saved to /home/swayam/projects/forest_fire_spread/fire-probability-prediction-map-unstacked-data/dataset_unstacked/fuel_map/resampled_uk_fuel_map_2020.tif
Resampled GHSL dataset saved to /home/swayam/projects/forest_fire_spread/fire-probability-prediction-map-unstacked-data/dataset_unstacked/ghsl/resampled_ghsl_uk_urbanmask2015.tif


## **Step 3.2: Resample Weather and Fire Mask (Relatively safe to run, RUN ONLY ONCE) (Not required anymore)**

As we are using resampled data directly

In [ ]:
# import logging

# logging.basicConfig(
#     level=logging.INFO,
#     format='%(asctime)s - %(levelname)s - %(message)s',
#     handlers=[
#         logging.FileHandler('resample_log.txt'),
#         logging.StreamHandler()
#     ]
# )
# logger = logging.getLogger(__name__)

# # weather_dir = os.path.join(base_dir, 'weather')
# # fire_mask_dir = os.path.join(base_dir, 'fire_mask')
# resampled_weather_dir = os.path.join(base_dir, 'resampled_weather')
# resampled_fire_mask_dir = os.path.join(base_dir, 'resampled_fire_mask')

# os.makedirs(resampled_weather_dir, exist_ok=True)
# os.makedirs(resampled_fire_mask_dir, exist_ok=True)

# weather_files = sorted(glob.glob(f"{weather_dir}/era5_2016_*.tif"))


# if not weather_files:
#     print("No weather files found!")
#     exit()

# # Get list of date strings
# date_strs = [os.path.basename(wf).replace('era5_2016_', '').replace('.tif', '') for wf in weather_files]
# logger.info(f"Found {len(date_strs)} weather files to process.")

# # Loop to process data
# for date_str in date_strs:
#     wf = os.path.join(weather_dir, f'era5_2016_{date_str}.tif')
#     fire_path = os.path.join(fire_mask_dir, f'fire_mask_2016_{date_str}.tif')

#     logger.info(f"Processing date: {date_str}")
    
#     if not os.path.exists(fire_path):
#         logger.warning(f"Skipping {date_str}: fire mask missing at {fire_path}")
#         continue
    
#     # Resample weather data (5 bands) (if not already resampled)
#     weather_output_path = os.path.join(resampled_weather_dir, f'resampled_weather_{date_str}.tif')

#     if not os.path.exists(weather_output_path):
#         try:
#             logger.info(f"Resampling weather data: {wf}")
#             with rasterio.open(wf) as w_src:
#                 weather_resampled = np.zeros((5, ref_shape[0], ref_shape[1]), dtype=np.float32)
#                 for band in range(1, 6):
#                     reproject(
#                         source=rasterio.band(w_src, band),
#                         destination=weather_resampled[band-1],
#                         src_transform=w_src.transform,
#                         src_crs=w_src.crs,
#                         dst_transform=ref_transform,
#                         dst_crs=ref_crs,
#                         resampling=Resampling.bilinear
#                     )
#                 # Save resampled weather data
#                 weather_meta = w_src.meta.copy()
#                 weather_meta.update({
#                     'count': 5,
#                     'dtype': 'float32',
#                     'height': ref_shape[0],
#                     'width': ref_shape[1],
#                     'transform': ref_transform,
#                     'crs': ref_crs
#                 })
                
#                 with rasterio.open(weather_output_path, 'w', **weather_meta) as dst:
#                     dst.write(weather_resampled)
#                 logger.info(f"Successfully saved resampled weather data: {weather_output_path}")
#         except Exception as e:
#             logger.error(f"Failed to resample weather data for {date_str}: {str(e)}")
#             continue
    
#     # Resample fire mask (1 band) (if not already resampled)
#     fire_output_path = os.path.join(resampled_fire_mask_dir, f'resampled_fire_mask_{date_str}.tif')

#     if not os.path.exists(fire_output_path):
#         try:
#             logger.info(f"Resampling fire mask: {fire_path}")
#             with rasterio.open(fire_path) as fm_src:
#                 fire_resampled = np.zeros(ref_shape, dtype=np.float32)
#                 reproject(
#                     source=rasterio.band(fm_src, 1),
#                     destination=fire_resampled,
#                     src_transform=fm_src.transform,
#                     src_crs=fm_src.crs,
#                     dst_transform=ref_transform,
#                     dst_crs=ref_crs,
#                     resampling=Resampling.nearest
#                 )
#                 # Save resampled fire mask
#                 fire_meta = fm_src.meta.copy()
#                 fire_meta.update({
#                     'dtype': 'float32',
#                     'height': ref_shape[0],
#                     'width': ref_shape[1],
#                     'transform': ref_transform,
#                     'crs': ref_crs
#                 })
                
#                 with rasterio.open(fire_output_path, 'w', **fire_meta) as dst:
#                     dst.write(fire_resampled, 1)
#                 logger.info(f"Successfully saved resampled fire mask: {fire_output_path}")
#         except Exception as e:
#             logger.error(f"Failed to resample fire mask for {date_str}: {str(e)}")
#             continue

# logger.info("Processing complete.")

## **Step 3.2: Resample Weather and Fire Mask Using Multiprocessing (VERY DANGEROUS, RUN ONLY ONCE, AT YOUR OWN RISK) (Not required anymore)**

In [15]:
# from multiprocessing import Pool, cpu_count

# # weather_dir = os.path.join(base_dir, 'weather')
# # fire_mask_dir = os.path.join(base_dir, 'fire_mask')
# resampled_weather_dir = os.path.join(base_dir, 'resampled_weather')
# resampled_fire_mask_dir = os.path.join(base_dir, 'resampled_fire_mask')

# os.makedirs(resampled_weather_dir, exist_ok=True)
# os.makedirs(resampled_fire_mask_dir, exist_ok=True)

# weather_files = sorted(glob.glob(f"{weather_dir}/era5_2016_*.tif"))

# if not weather_files:
#     print("No weather files found!")
#     exit()

# # Function to process datas
# def process_dataset(date_str):
#     wf = os.path.join(weather_dir, f'era5_2016_{date_str}.tif')
#     fire_path = os.path.join(fire_mask_dir, f'fire_mask_2016_{date_str}.tif')
    
#     if not os.path.exists(fire_path):
#         print(f"Skipping {date_str}: fire mask missing")
#         return
    
#     # Resample weather data (5 bands) (if not already resampled)
#     weather_output_path = os.path.join(resampled_weather_dir, f'resampled_weather_{date_str}.tif')

#     if not os.path.exists(weather_output_path):
#         with rasterio.open(wf) as w_src:
#             weather_resampled = np.zeros((5, ref_shape[0], ref_shape[1]), dtype=np.float32)
#             for band in range(1, 6):
#                 reproject(
#                     source=rasterio.band(w_src, band),
#                     destination=weather_resampled[band-1],
#                     src_transform=w_src.transform,
#                     src_crs=w_src.crs,
#                     dst_transform=ref_transform,
#                     dst_crs=ref_crs,
#                     resampling=Resampling.bilinear
#                 )
#             # Save resampled weather data
#             weather_meta = w_src.meta.copy()
#             weather_meta.update({
#                 'count': 5,
#                 'dtype': 'float32',
#                 'height': ref_shape[0],
#                 'width': ref_shape[1],
#                 'transform': ref_transform,
#                 'crs': ref_crs
#             })
            
#             with rasterio.open(weather_output_path, 'w', **weather_meta) as dst:
#                 dst.write(weather_resampled)
    
#     # Resample fire mask (1 band) (if not already resampled)
#     fire_output_path = os.path.join(resampled_fire_mask_dir, f'resampled_fire_mask_{date_str}.tif')

#     if not os.path.exists(weather_output_path):
#         with rasterio.open(fire_path) as fm_src:
#             fire_resampled = np.zeros(ref_shape, dtype=np.float32)
#             reproject(
#                 source=rasterio.band(fm_src, 1),
#                 destination=fire_resampled,
#                 src_transform=fm_src.transform,
#                 src_crs=fm_src.crs,
#                 dst_transform=ref_transform,
#                 dst_crs=ref_crs,
#                 resampling=Resampling.nearest
#             )
#             # Save resampled fire mask
#             fire_meta = fm_src.meta.copy()
#             fire_meta.update({
#                 'dtype': 'float32',
#                 'height': ref_shape[0],
#                 'width': ref_shape[1],
#                 'transform': ref_transform,
#                 'crs': ref_crs
#             })
            
#             with rasterio.open(fire_output_path, 'w', **fire_meta) as dst:
#                 dst.write(fire_resampled, 1)

# # Get list of date strings
# date_strs = [os.path.basename(wf).replace('era5_2016_', '').replace('.tif', '') for wf in weather_files]

# # Use half of the available cores (6 on a 12-thread CPU)
# num_cores = cpu_count() // 2
# with Pool(num_cores) as p:
#     p.map(process_dataset, date_strs)

## **Stacking the 5 datasets with 10 bands and saving separately for each each**

In [ ]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('resample_log.txt'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# weather_dir = os.path.join(base_dir, 'weather')
# fire_mask_dir = os.path.join(base_dir, 'fire_mask')
resampled_weather_dir = os.path.join(base_dir, 'resampled_weather')
resampled_fire_mask_dir = os.path.join(base_dir, 'resampled_fire_mask')

os.makedirs(resampled_weather_dir, exist_ok=True)
os.makedirs(resampled_fire_mask_dir, exist_ok=True)

weather_files = sorted(glob.glob(f"{weather_dir}/era5_2016_*.tif"))

nodata_value = -9999

if not weather_files:
    print("No weather files found!")
    exit()

# Get list of date strings
date_strs = [os.path.basename(wf).replace('era5_2016_', '').replace('.tif', '') for wf in weather_files]
logger.info(f"Found {len(date_strs)} weather files to process.")

# Loop to process data
for date_str in date_strs:
    weather_path = os.path.join(weather_dir, f'era5_2016_{date_str}.tif')
    fire_path = os.path.join(fire_mask_dir, f'fire_mask_2016_{date_str}.tif')

    logger.info(f"Processing date: {date_str}")

    if not os.path.exists(weather_path):
        logger.warning(f"Skipping {date_str}: weather missing at {weather_path}")
        continue
        
    if not os.path.exists(fire_path):
        logger.warning(f"Skipping {date_str}: fire mask missing at {fire_path}")
        continue

    # Load weather data (5 bands)
    try:
        logger.info(f"Loading weather data: {weather_path}")
        with rasterio.open(weather_path) as w_src:
            weather = w_src.read()  # Read all 5 bands
            if weather.shape[0] != 5 or weather.shape[1:] != ref_shape:
                raise ValueError(f"Weather data shape {weather.shape} does not match reference shape {ref_shape}")
            logger.info(f"Successfully loaded weather data: {weather_path}")
    except Exception as e:
        logger.error(f"Failed to load resampled weather data for {date_str}: {str(e)}")
        continue
    
    # Load fire mask (1 band)
    try:
        logger.info(f"Loading fire mask: {fire_path}")
        with rasterio.open(fire_path) as fm_src:
            fire = fm_src.read(1)  # Read single band
            if fire.shape != ref_shape:
                raise ValueError(f"Fire mask shape {fire.shape} does not match reference shape {ref_shape}")
            logger.info(f"Successfully loaded fire mask: {fire_path}")
    except Exception as e:
        logger.error(f"Failed to load resampled fire mask for {date_str}: {str(e)}")
        continue

    # Stack and save with logging
    try:
        logger.info(f"Stacking data for date: {date_str}")
        
        # Fill masked arrays
        slope_filled = slope.filled(nodata_value)
        aspect_filled = aspect.filled(nodata_value)
        fuel_filled = fuel.filled(nodata_value)
        ghsl_filled = ghsl.filled(nodata_value)
        fire_filled = fire  # Already a regular array

        # Stack into one array (10 bands)
        stacked = np.stack([
            *weather,
            slope_filled,
            aspect_filled,
            fuel_filled,
            ghsl_filled,
            fire_filled
        ])

        # Update metadata
        ref_meta.update({
            "count": 10,
            "dtype": 'float32',
            "nodata": nodata_value
        })

        # Save
        out_path = os.path.join(output_dir, f'stack_2016_{date_str}.tif')
        with rasterio.open(out_path, 'w', **ref_meta) as dst:
            dst.write(stacked.astype('float32'))
        
        logger.info(f"Successfully saved stacked raster: {out_path}")
        print(f"✅ Saved: {out_path}")
    except Exception as e:
        logger.error(f"Failed to stack or save data for {date_str}: {str(e)}")
        continue

    w_src.close()
    fm_src.close()

s.close()
a.close()
f.close()
g.close()

logger.info("Processing complete.")